In [ ]:
!git clone https://github.com/zackproser/portfolio.git

In [ ]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [ ]:
loader = DirectoryLoader('portfolio', glob="**/*.mdx")

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

index_name = "pinecone-chatbot"

# Split our documents into chunks
text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)

In [ ]:
split_docs[25]

In [ ]:
vectorstore = PineconeVectorStore.from_documents(split_docs, embeddings, index_name=index_name)

In [ ]:
query = "What is the programming bug?"

similar_docs = vectorstore.similarity_search(query)

In [ ]:
similar_docs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa.invoke(query)